# Store Item Demand Forecasting - Model Training
This is the second notebook on this project where I'll be running a data modeling process on the product sales dataset available at the Kaggle's *Store Item Demand Forecasting Challende*.

Here I'll be training Facebook's Prophet different models at scale, to cover all the products and stores.

<img src="https://i.ibb.co/FDsQbZX/kaggle-comp-banner.png" width="900" />

**Challenge Description**: <br>

<i>
This competition is provided as a way to explore different time series techniques on a relatively simple and clean dataset.

You are given 5 years of store-item sales data, and asked to predict 3 months of sales for 50 different items at 10 different stores.

What's the best way to deal with seasonality? Should stores be modeled separately, or can you pool them together? Does deep learning work better than ARIMA? Can either beat xgboost?

This is a great competition to explore different models and improve your skills in forecasting.
</i>

**Challenge Goal**: <br>

<i>
The objective of this competition is to predict 3 months of item-level sales data at different store locations. (with no holiday effect)
</i>

**Author**: Arthur G.

## Loading Dependencies
Here I'll be loading and setting up the dependencies for this notebook.

In [1]:
# add custom functions
import sys
sys.path.append('../')

# add libs
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from fbprophet import Prophet
import matplotlib.pyplot as plt
from fbprophet.serialize import model_to_json
from src.visualization.visualize import model_performance_indicators

# setting up libs
warnings.filterwarnings('ignore')
seed = np.random.seed(42)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)
plt.style.use('ggplot')
plt.rc('font', **{'family': 'DejaVu Sans', 'size': 22})

## Loading Data
Now it's time to load the dataset.

In [2]:
sales_df = pd.read_csv(os.path.join('..', 'data', 'raw', 'train.csv'))
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB


## Data Preparation
Here I'll be preparing the data for the modeling-at-scale process. Let's start by grouping at store_item level.

In [3]:
# adjusting column names to match Prophet requirements
sales_df.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

# grouping data
modeling_groups = sales_df.groupby(['store', 'item'])
modeling_groups_keys = list(modeling_groups.groups.keys())

## Training with XGBoost
This is a hybrid approach where I'm using XGBoost to improve the capacity of our model to "understand" non-stationary components.

Let's start by training the prophet model.

In [4]:
# selecting one combination for training
store_item_data = modeling_groups.get_group((2, 15)).drop(columns=['store', 'item'])

# train-test split
train_data = store_item_data.iloc[:-91, :]
test_data = store_item_data.iloc[-91:, :]

# creating a prophet model
model = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='multiplicative'
)

# model fitting
model.fit(train_data)

Initial log joint probability = -16.7167
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        4317.7   0.000241727       131.801      0.3729           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     114       4317.94   0.000113664       98.1512   3.657e-07       0.001      185  LS failed, Hessian reset 
     167       4318.45   0.000162663       159.036   1.933e-06       0.001      287  LS failed, Hessian reset 
     199       4318.53   1.93851e-06       67.7111      0.3429      0.3429      332   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     270        4318.6   5.26354e-06       78.7335   6.307e-08       0.001      499  LS failed, Hessian reset 
     299       4318.64   0.000275228       75.4504           1           1      529   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Note

Now let's make future predictions in order to get data to traing XGBoost.

In [5]:
# forecasting the future three months
future = model.make_future_dataframe(periods=len(test_data), freq='D')
forecast = model.predict(future)

Merging Prophet components with XGBoost train data.

In [6]:
# getting prophet seasonal and trend components
prophet_components = forecast.loc[:, ['trend', 'daily', 'weekly', 'yearly']]

# merging prophet components as training data
xgb_train_data = store_item_data.copy()
xgb_train_data['trend'] = prophet_components.trend.values
xgb_train_data['daily'] = prophet_components.daily.values
xgb_train_data['weekly'] = prophet_components.weekly.values
xgb_train_data['yearly'] = prophet_components.yearly.values

xgb_train_data.head()

,ds,y,trend,daily,weekly,yearly
257466,2013-01-01,65,89.013,-0.009,-0.081,-0.276
257467,2013-01-02,71,89.048,-0.009,-0.078,-0.276
257468,2013-01-03,53,89.084,-0.009,-0.006,-0.277
257469,2013-01-04,68,89.119,-0.009,0.052,-0.278
257470,2013-01-05,77,89.154,-0.009,0.127,-0.279


Let's do train-test split and build up XGBoost data matrices.

In [7]:
# train-test split
fbp_xgb_train_set = xgb_train_data.iloc[:-91, :]
fbp_xgb_test_set = xgb_train_data.iloc[-91:, :]

# predictors-target split
x_train = fbp_xgb_train_set.drop(columns='y').iloc[:, 1:]
x_test = fbp_xgb_test_set.drop(columns='y').iloc[:, 1:]

y_train = fbp_xgb_train_set.y
y_test = fbp_xgb_test_set.y

# train-test XGBoost DMatrix
train_dmatrix = xgb.DMatrix(
    data=x_train,
    label=y_train
)
test_dmatrix = xgb.DMatrix(
    data=x_test,
    label=y_test
)

Let's finally train our XGBoost model.

In [8]:
# setting parameters
xgb_parameters = {
    'learning_rate': 0.1,
    'max_depth': 3,
    'colsample_bytree': 1,
    'subsample': 1,
    'min_child_weight': 1,
    'gamma': 1,
    'random_state': seed,
    'eval_metric': 'rmse',
    'objective': 'reg:squarederror'
}

model_xgb = xgb.train(
    params=xgb_parameters,
    dtrain=train_dmatrix,
    num_boost_round=100,
    evals=[(test_dmatrix, 'y')],
    verbose_eval=15
)

[0]	y-rmse:107.65150
[15]	y-rmse:30.49649
[30]	y-rmse:13.49310
[45]	y-rmse:11.75148
[60]	y-rmse:11.41088
[75]	y-rmse:11.32943
[90]	y-rmse:11.32506
[99]	y-rmse:11.32215


Let's consider a RMSE of 11, for a three-months prediction range a good fit, although it definitely could be improved. 